In [ ]:
from datetime import datetime
  
from pytz import timezone
tz = timezone('EST')
print(datetime.now(tz)) 

2022-05-10 13:48:06.670973-05:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
!pip uninstall -y torch
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
#print(torch.__version__)
def format_pytorch_version(version):
  return version.split('+')[0]

import torch
import numpy as np
import gc

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 
!pip install torch ninja
!pip install -U MinkowskiEngine --install-option="--blas=openblas" -v --no-deps

Found existing installation: torch 1.11.0+cu113
Uninstalling torch-1.11.0+cu113:
  Successfully uninstalled torch-1.11.0+cu113
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████▌                   | 834.1 MB 1.4 MB/s eta 0:16:05tcmalloc: large alloc 1147494400 bytes == 0x3a4a4000 @  0x7f612785b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████▉                | 1055.7 MB 1.3 MB/s eta 0:13:48tcmalloc: large alloc 1434370048 bytes == 0x7eafa000 @  0x7f612785b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0

In [ ]:
tz = timezone('EST')
print(datetime.now(tz)) 

2022-05-10 14:06:49.806204-05:00


In [ ]:
import torch
from torch_geometric.datasets import S3DIS
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

#dataset = S3DIS(root='/content/drive/MyDrive/CS674_Project/Dataset/')

True
Tesla T4


In [ ]:
from torch_geometric.datasets import S3DIS
train_dataset = S3DIS(
    '/content/',
    test_area=5, 
    train=True)

test_dataset = S3DIS(
    '/content/',
    test_area=5, 
    train=False)

Extracting /content/indoor3d_sem_seg_hdf5_data.zip
Processing...
Done!


In [ ]:
bs = 5
from torch_geometric.loader import DataLoader
loader = DataLoader(train_dataset, batch_size=bs)
from torch_geometric.loader import DataLoader
testloader = DataLoader(test_dataset, batch_size=bs)

In [ ]:
len_test = len(testloader) - 1
print(len_test)
batch = next(iter(loader))
print(batch['batch'].shape[0])

import numpy as np
np.random.seed(42)
val_indices = np.random.choice(len_test, int(len_test * 0.3))
# test_indices = [i for i in range(6851) if i not in val_indices]
test_set, val_set = [], []
for i, batch in enumerate(testloader):
  # if i > 10:
  #   break
  # print(batch['x'][0])
  if i in val_indices:
    val_set.append(batch)
  else:
    test_set.append(batch)
# print(i)

#batch['batch']

1370
20480


In [ ]:
import numpy as np
np.random.seed(42)
val_indices = np.random.choice(len_test, int(len_test * 0.3))
# test_indices = [i for i in range(6851) if i not in val_indices]
test_set, val_set = [], []
for i, batch in enumerate(testloader):
  # if i > 10:
  #   break
  # print(batch['x'][0])
  if i in val_indices:
    val_set.append(batch)
  else:
    test_set.append(batch)
# print(i)

In [ ]:
import MinkowskiEngine as ME
import torch
import MinkowskiEngine.MinkowskiFunctional as MF
import MinkowskiEngine.MinkowskiNonlinearity as MN


In [ ]:
from torch import nn
class NonLocalModule(nn.Module):
    def __init__(self):
        super(NonLocalModule, self).__init__()
        
        self.MHA = nn.MultiheadAttention(6,1, batch_first=True)
        self.linear = nn.Linear(6,16)


    def forward(self, query, key, val):
        

        query = query.unsqueeze(0)
        key = key.unsqueeze(0)
        val = val.unsqueeze(0)
        #print('In Attn ', query.size())
        attn_op, _ = self.MHA(query, key, val)
        #print("attention op: ", attn_op.size())
        out = nn.functional.relu(self.linear(attn_op))
        #print("out from attention: ", out.size())

        return out

In [ ]:
from torch.nn import functional as F

class ResUNet(ME.MinkowskiNetwork):

    def __init__(self, in_nchannel, out_nchannel, D):
        super(ResUNet, self).__init__(D)
        

        self.query = ME.MinkowskiLinear(6,6)
        self.key = ME.MinkowskiLinear(6,6)
        self.value = ME.MinkowskiLinear(6,6)

        self.attn = NonLocalModule()

        self.conv_n_1 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=in_nchannel,
                out_channels=32,
                kernel_size=1,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(32),   # maybe not needed 
        )
        
        self.res_block_1 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(32),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(32),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        self.conv_1 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.res_block_2 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(64),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(64),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        self.conv_2 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.res_block_3 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(128),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(128),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        self.conv_3 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=256,
                kernel_size=3,
                stride=2,
                dimension=D))
        

        # downsample

        self.upsample_3 = torch.nn.Sequential(
            ME.MinkowskiConvolutionTranspose(
                in_channels=256,
                out_channels=128,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.conv_n_2 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=256,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(128),
        )
        
        self.res_block_4 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(128),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(128),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        #
        self.upsample_2 = torch.nn.Sequential(
            ME.MinkowskiConvolutionTranspose(
                in_channels=128,
                out_channels=64,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.conv_n_3 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(64),
        )
        
        self.res_block_5 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(64),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(64),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        #
        self.upsample_1 = torch.nn.Sequential(
            ME.MinkowskiConvolutionTranspose(
                in_channels=64,
                out_channels=32,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.conv_n_4 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(32),
        )
        
        self.res_block_6 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(32),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(32),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        # final layer
        
        self.conv_4 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=out_nchannel,
                kernel_size=1,
                stride=1,
                dimension=D))

        

    def forward(self, input, coords):
        
        #print("input to ResUNet: ", input.size())

        query = self.query(input)
        key = self.key(input)
        val = self.value(input)

        #print("query, key, val", query.size(), key.size(), val.size())
        #print(query)

        query = query.features
        key = key.features
        val = val.features

        #print("query, key, val", query.size(), key.size(), val.size())

        # remaining = 4096 - query.shape[0]

        # query = torch.nn.functional.pad(query, (0, 0, 0, remaining), mode='constant', value=0).to('cuda:0')
        # key = torch.nn.functional.pad(key, (0, 0, 0, remaining), mode='constant', value=0).to('cuda:0')
        # val = torch.nn.functional.pad(val, (0, 0, 0, remaining), mode='constant', value=0).to('cuda:0')

        # print("query, key, val", query.size(), key.size(), val.size()) #4096, 6

        out = self.attn(query, key, val)
        out = torch.squeeze(out)
        #print("out from attn module in ResUNet", out.size()) #input.shape[0], 16

        input = ME.SparseTensor(out, coords, device=device)

        #print("conversion to sparse tensor: ", input.size(), type(input))

        out = self.conv_n_1(input)
        #print("out ", out.size(), type(input))
        out_1 = self.res_block_1(out)
        #print(out_1.size())

        out = self.conv_1(out_1)
        out_2 = self.res_block_2(out)
        #print(out_2.size())

        out = self.conv_2(out_2)
        # dense_1 = torch.bmm(ME.dense(out))
        out_3 = self.res_block_3(out)
        #print(out_3.size())

        out = self.conv_3(out_3)
        out = self.upsample_3(out)
        out = ME.cat(MF.relu(out_3), out)
        #print(out.size())

        out = self.conv_n_2(out)
        out = self.res_block_4(out)
        out = self.upsample_2(out)
        out = ME.cat(MF.relu(out_2), out)
        #print(out.size())

        out = self.conv_n_3(out)
        out = self.res_block_5(out)
        out = self.upsample_1(out)
        out = ME.cat(MF.relu(out_1), out)
        #print(out.size())

        out = self.conv_n_4(out)
        out = self.res_block_6(out)
        out = self.conv_4(out)
        #print("out in ResUNet:", out.size())
        # outputs_sliced = out.slice(input)
        # len(outputs_sliced) == len(coords)
        #print("outputs sliced, out", outputs_sliced.shape, out.shape)
        #print("outputs slice features, out", outputs_sliced.F.shape, out.shape)

        return out

In [ ]:
torch.cuda.empty_cache()


In [ ]:
#train cell
import gc
#torch.cuda.empty_cache()
gc.collect()
from torch._C import device
import torch.utils.data as utils_data
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import jaccard_score

from torch.nn import functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model     = ResUNet(16, 13, D= 3).to(device)
criterion = torch.nn.CrossEntropyLoss()
learningRate = 0.00001
numEpochs    = 10
weightDecay  = 0.01
optimizer    = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)
val_loss_list = []
train_loss_list = []
miou_list = []

for epoch in range(numEpochs):
       
    train_loss  = 0
    print("epoch", epoch)
    model.train()
    for i, batch in enumerate(loader):

        torch.cuda.empty_cache()

        #print("batch", i)
       
        pos_batch, x_batch, y_batch = batch['pos'], batch['x'], batch['y']
        #print(pos_batch[:10], x_batch[:10])
        print("pos_batch", pos_batch.shape)
        
        pos_batch = torch.cat((torch.ones(pos_batch.size()[0], 1)*i, pos_batch),1)
        #print("pos batch: ", pos_batch.size())
       
        coords, feats, labels = ME.utils.sparse_quantize(pos_batch.cpu(), x_batch.cpu(), y_batch.cpu(), quantization_size=0.1, device='cuda')
        #print("coords, feats, labels:", coords.size(), feats.size(), labels.size())
        
        input = ME.SparseTensor(feats, coords, device=device)
        #print(input)

        # input = ME.TensorField(coordinates=coords, features=feats, quantization_mode=ME.MinkowskiTensor.SparseTensorQuantizationMode.UNWEIGHTED_AVERAGE)
        # sinput = input.sparse()
        # optimizer.zero_grad()
        # outputs = model(sinput, coords)
        # outputs_sliced = outputs.slice(input)
        # len(outputs_sliced) == pos_batch.shape[0]
        # print("outputs sliced, outputs", outputs_sliced.shape, outputs.shape)
        
        #print("outputs_sliced, outputs" ,outputs_sliced.shape, outputs.shape)
        #print(type(outputs))
        outputs=outputs.F
        #print("outputs, labels: ", outputs.size(), labels.size())
        

        loss = criterion(outputs.to(device), labels.long().to(device))
        train_loss +=  loss.item()
        if i%200 == 0:
          print('Loss for epoch', epoch, 'and batch', i, 'is', loss.item())

        loss.backward()
        optimizer.step()
        #break

        # if(i == 1):
        #   break

    train_loss /= (i+1)
    #model.eval()

    val_loss = 0
    with torch.no_grad():
      js = 0
      for i, batch in enumerate(val_set):

        pos_batch, x_batch, y_batch = batch['pos'], batch['x'], batch['y']
        
        pos_batch = torch.cat((torch.ones(pos_batch.size()[0], 1)*i, pos_batch),1)
       

        #print(pos_batch.shape, x_batch.shape, y_batch.shape)
        
        
        coords, feats, labels = ME.utils.sparse_quantize(pos_batch.cpu(), x_batch.cpu(), y_batch.cpu(), quantization_size=0.1, device='cuda')
        input = ME.SparseTensor(feats, coords, device=device)
        # labels = y_batch
        # input = ME.SparseTensor(x_batch, pos_batch, device=device)
        #input = ME.TensorField(coordinates=x_batch, features=x_batch, quantization_mode=ME.SparseTensorQuantizationMode.UNWEIGHTED_AVERAGE)
        #print(len(input))
        #input = input.sparse()
        #print("inputs.size, x_batch: ", input.size(), x_batch.size())
        #print(input.size(), labels.long().size())
        # forward pass
        outputs = model(input, coords)

        outputs = outputs.F
        loss = criterion(outputs.to(device), labels.long().to(device))
        val_loss +=  loss.item()

        indices = torch.argmax(F.softmax(outputs,dim=1), dim=1).cpu()
        lbl = labels.cpu()
        js += jaccard_score(lbl, indices, labels=np.arange(13), average='weighted', zero_division=0.0)


        # outputs = outputs.slice(input) #(this requireds original ME.sparsetensor that discretized the input.. here it was input)
        # len(outputs) == len(pos_batch)
        #print("outputs.size: ", outputs.size())
        # outputs=outputs.F
        #ME.MinkowskiToDenseTensor((4096,))(outputs) #not needed anywhere
        # print(outputs.size())
        # smax = F.softmax(outputs, dim=1)
        # print(smax.size())
        #print(outputs.size(), labels.size())
        # indices = torch.argmax(smax, dim=1).cpu()
        # js += jaccard_score(labels, indices, labels=np.arange(12), average='weighted', zero_division=0.0)
        
        # if(i == 1):
        #   break
    
    if(epoch == 5):
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/model5_attention.pth')
    
    val_loss /= (i+1)
    print('Epoch:', epoch, '| Train loss:', train_loss, '| Validation loss:', val_loss, '| mIoU:', js/(i+1))
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    miou_list.append(js/(i+1))
        
    # print(train_loss)
            
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/model10_attention.pth')          


epoch 0
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([829, 13]) torch.Size([829, 13])
Loss for epoch 0 and batch 0 is 2.5810768604278564
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([1329, 13]) torch.Size([1329, 13])
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([404, 13]) torch.Size([404, 13])
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([1051, 13]) torch.Size([1051, 13])
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([930, 13]) torch.Size([930, 13])
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([422, 13]) torch.Size([422, 13])
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([1015, 13]) torch.Size([1015, 13])
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([753, 13]) torch.Size([753, 13])
pos_batch torch.Size([20480, 3])
outputs sliced, outputs torch.Size([818, 13]) torch.Size([818, 13])
pos_batch torch.Size([2048

KeyboardInterrupt: ignored

In [ ]:
print("train: ", train_loss_list)
print("val: ", val_loss_list)
print("miou list: ", miou_list)

train:  [1.6590031662342049, 1.2835031663406276, 1.1596191964079667, 1.0786134538131933, 1.0156946693618583, 0.962801229781915, 0.917019992234246, 0.8772889522516731, 0.8402102571754339, 0.8061184184917913]
val:  [1.511029231202775, 1.374601403699405, 1.3045632008193195, 1.2625380385613096, 1.235007150363231, 1.208986912343813, 1.1912368748499, 1.1813174275384433, 1.1770575725513956, 1.1767086613869322]
miou list:  [0.45208710796772195, 0.4862443943355026, 0.49672075382430736, 0.5034151459936733, 0.5095022633422536, 0.5210455359431909, 0.5299378544029097, 0.5379425482712697, 0.5435940512858547, 0.5474066137448375]
